In [ ]:
#실험파일
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
from tkinter import *
from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
import sys
from os import path
from flask import Flask
from collections import deque


app = Flask(__name__)

@app.route('/') 

#웹페이지 템플릿
def home(): 
    return '''
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <title>Web Cam Video</title>
            <style>
            H1 {
              position: absolute;
              left: 100px;
              top: 15px;
              color:black;
            }
            </style>

            <H1>Webcam</H1>
            </head>
            <body>
                  <div class="booth">
                      <video id="video" width="400" height="300" autoplay=""></video>
                      <canvas id="canvas" width="400" height="300"></canvas>
            </div>
                  <script src="./javascripts/video.js"></script>
            </body>
            </html>

            <style>
            H2 {
              position: absolute;
              left: 120px;
              top: 100px;
              color:red;
            }
            </style>

            <H2><form action="/webcam/" method="post"><button name="forwardBtn" type="submit">webcam.exe</button></H2></form>'''

@app.route("/webcam/", methods=['POST'])

#메인함수
def Web_cam():
    #UI파일 연결
    #단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
    form_class = uic.loadUiType("controller.ui")[0]
    c = 0
    s = 0
    q = 0

    record = False
    rs = False
    r = False
    res = True

    #화면을 띄우는데 사용되는 Class 선언
    class WindowClass(QMainWindow, form_class) :

        def __init__(self) :

            super().__init__()
            self.setupUi(self)

            #버튼에 기능을 연결하는 코드
            self.pushButton.clicked.connect(self.Webcams)
            self.pushButton_2.clicked.connect(self.Backs)
            self.pushButton_3.clicked.connect(self.Nexts)
            self.pushButton_4.clicked.connect(self.Resets)
            self.pushButton_5.clicked.connect(self.Captures)
            self.pushButton_6.clicked.connect(self.Records)
            self.pushButton_7.clicked.connect(self.Stop_Records)
            self.pushButton_8.clicked.connect(self.Exit)
            
        #웹캠기능
        def Webcams(self):
            global c
            global s
            global q
            global record
            global rs
            global r
            global res
            
            q = 0
            c = 0
            s = 0
            
            record = False
            rs = False
            r = False
            res = True
            
            #자동추적 펜 인식 그림 그리기 기능 세팅
            # Define the upper and lower boundaries for a color to be considered "Blue"
            blueLower = np.array([100, 60, 60])
            blueUpper = np.array([140, 255, 255])

            # Define a 5x5 kernel for erosion and dilation
            kernel = np.ones((5, 5), np.uint8)

            # Setup deques to store separate colors in separate arrays
            bpoints = [deque(maxlen=512)]
            gpoints = [deque(maxlen=512)]
            rpoints = [deque(maxlen=512)]
            ypoints = [deque(maxlen=512)]

            bindex = 0
            gindex = 0
            rindex = 0
            yindex = 0

            colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
            colorIndex = 0

            # Setup the Paint interface
            #paintWindow = np.zeros((471,636,3)) + 255
            #paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
            #paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), colors[0], -1)
            #paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), colors[1], -1)
            #paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), colors[2], -1)
            #paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), colors[3], -1)
            
            paintWindow = np.zeros((471,636,3)) + 255
            paintWindow = cv2.rectangle(paintWindow, (40,1), (80,65), (0,0,0), 2)
            paintWindow = cv2.rectangle(paintWindow, (90,1), (130,65), colors[0], -1)
            paintWindow = cv2.rectangle(paintWindow, (140,1), (150,65), colors[1], -1)
            paintWindow = cv2.rectangle(paintWindow, (160,1), (200,65), colors[2], -1)
            paintWindow = cv2.rectangle(paintWindow, (210,1), (250,65), colors[3], -1)
            
            cv2.putText(paintWindow, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)

            ###cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

            fourcc = cv2.VideoWriter_fourcc(*'XVID')

            logo = cv2.imread("./zero.png")

            cap = cv2.VideoCapture(0)

            cap.set(10000, 10000)

            time.sleep(3) #warming up
            if not cap.isOpened():
                exit()

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

            while True:    

                # Grab the current paintWindow
                #(grabbed, frame) = cap.read()
                ret, image = cap.read()
                image = cv2.flip(image, 1)
                hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

                # Add the coloring options to the frame
                image = cv2.rectangle(image, (40,1), (140,65), (122,122,122), -1)
                image = cv2.rectangle(image, (160,1), (255,65), colors[0], -1)
                image = cv2.rectangle(image, (275,1), (370,65), colors[1], -1)
                image = cv2.rectangle(image, (390,1), (485,65), colors[2], -1)
                image = cv2.rectangle(image, (505,1), (600,65), colors[3], -1)
                cv2.putText(image, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)

                # Check to see if we have reached the end of the video
                if not ret:
                    break

                # Determine which pixels fall within the blue boundaries and then blur the binary image
                blueMask = cv2.inRange(hsv, blueLower, blueUpper)
                blueMask = cv2.erode(blueMask, kernel, iterations=2)
                blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
                blueMask = cv2.dilate(blueMask, kernel, iterations=1)

                # Find contours in the image
                (cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
                center = None

                # Check to see if any contours were found
                if len(cnts) > 0:
                    # Sort the contours and find the largest one -- we
                    # will assume this contour correspondes to the area of the bottle cap
                    cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
                    # Get the radius of the enclosing circle around the found contour
                    ((x, y), radius) = cv2.minEnclosingCircle(cnt)
                    # Draw the circle around the contour
                    cv2.circle(image, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                    # Get the moments to calculate the center of the contour (in this case Circle)
                    M = cv2.moments(cnt)
                    center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

                    if center[1] <= 65:
                        if 40 <= center[0] <= 140: # Clear All
                            bpoints = [deque(maxlen=512)]
                            gpoints = [deque(maxlen=512)]
                            rpoints = [deque(maxlen=512)]
                            ypoints = [deque(maxlen=512)]

                            bindex = 0
                            gindex = 0
                            rindex = 0
                            yindex = 0

                            paintWindow[67:,:,:] = 255
                        elif 160 <= center[0] <= 255:
                                colorIndex = 0 # Blue
                        elif 275 <= center[0] <= 370:
                                colorIndex = 1 # Green
                        elif 390 <= center[0] <= 485:
                                colorIndex = 2 # Red
                        elif 505 <= center[0] <= 600:
                                colorIndex = 3 # Yellow
                    else :
                        if colorIndex == 0:
                            bpoints[bindex].appendleft(center)
                        elif colorIndex == 1:
                            gpoints[gindex].appendleft(center)
                        elif colorIndex == 2:
                            rpoints[rindex].appendleft(center)
                        elif colorIndex == 3:
                            ypoints[yindex].appendleft(center)
                # Append the next deque when no contours are detected (i.e., bottle cap reversed)
                else:
                    bpoints.append(deque(maxlen=512))
                    bindex += 1
                    gpoints.append(deque(maxlen=512))
                    gindex += 1
                    rpoints.append(deque(maxlen=512))
                    rindex += 1
                    ypoints.append(deque(maxlen=512))
                    yindex += 1

                # Draw lines of all the colors (Blue, Green, Red and Yellow)
                points = [bpoints, gpoints, rpoints, ypoints]
                for i in range(len(points)):
                    for j in range(len(points[i])):
                        for k in range(1, len(points[i][j])):
                            if points[i][j][k - 1] is None or points[i][j][k] is None:
                                continue
                            cv2.line(image, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                            cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

                # Show the frame and the paintWindow image
                #cv2.imshow("Tracking", frame)
                #cv2.imshow("Paint", paintWindow)

                # If the 'q' key is pressed, stop the loop
                #if cv2.waitKey(1) & 0xFF == ord("q"):
               #     break


                #현재시간
                now = datetime.datetime.now().strftime("%d_%H-%M-%S")
                key = cv2.waitKey(3)

                #대체 이미지
                if(res==True):
                    logo = cv2.imread("./zero.png")
                else:
                    if (c == 1): 
                        logo = cv2.imread("./pick.png")
                    elif (c == 2): 
                        logo = cv2.imread("./smile.png")
                    elif (c == 3):
                        logo = cv2.imread("./keobi.png")
                    elif (c == 4):
                        logo = cv2.imread("./hahoi.png")
                    elif (c == 5):
                        logo = cv2.imread("./trumph.png")
                    elif (c == 6):
                        logo = cv2.imread("./Pineapple.png")
                    elif (c == 7):
                        logo = cv2.imread("./Watermelon.png")
                    elif (c == 8):
                        logo = cv2.imread("./Lion.png")
                    elif (c == 9):
                        logo = cv2.imread("./Neo.png")
                    elif (c == 10):
                        logo = cv2.imread("./Momo.png")
                    elif (c == 11):
                        logo = cv2.imread("./Prodo.png")
                    elif (c == 12):
                        logo = cv2.imread("./Muzi.png")

                faces = face_cascade.detectMultiScale(image)

                for face in faces:
                    (x, y, w, h) = face
                    x1, y1, x2, y2 = x, y, x + w, y + h
                    #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
                    
                    if(c == 0):
                        if(res == True):
                            logo = cv2.imread("./zero.png")
                        else:
                            image_face = image[y1:y2, x1:x2]
                            image_face = cv2.resize(image_face, (w//30, h//30))
                            image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                            image[y1:y2, x1:x2] = image_face
                            
                    else:
                        image_face = image[y1:y2, x1:x2]
                        logo = cv2.resize(logo, (w, h))
                        image_face =  cv2.add(image_face, logo)
                        image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                        image[y1:y2, x1:x2] = image_face

                    if(s == 1): 
                        cv2.imwrite("./" + str(now) + ".png", image)
                        s = 0

                    if(record == True):
                        record = False
                        r = True
                        video = cv2.VideoWriter("./" + str(now) + ".avi", fourcc, 30.0, (image.shape[1], image.shape[0]))

                    elif((r == True) and (rs == True)):
                        video.release()
                        record = False
                        rs = False
                        r = False

                    if(r == True):    
                        color = (0, 0, 255)
                        thickness = 2 
                        location = (550, 50)
                        font = cv2.FONT_HERSHEY_COMPLEX  # hand-writing style font
                        fontScale = 1.0
                        cv2.putText(image, 'REC', location, font, fontScale, color, thickness)
                        #print("녹화 중..")
                        video.write(image)


                    cv2.imshow("result", image)


                if (q == 1): #q
                    break

                if not ret:
                    break

            cap.release()
            cv2.destroyAllWindows()
        #다음 이미지
        def Nexts(self):
            global c
            if(c == 12):
                c = c
            else:
                c = c + 1
        #이전 이미지
        def Backs(self):
            global c
            if(c == 0):
                c = 0
            else:
                c = c - 1  
        #이미지 사용 여부
        def Resets(self):
            global res
            if(res == True):
                res = False
            else:
                res = True
        #캡쳐
        def Captures(self):
            global s
            s = 1
        #녹화 시작
        def Records(self):
            global record
            global r
            record = True
            r = True
        #녹화 중지
        def Stop_Records(self):
            global rs
            rs = True
            
        #웹캠 종료
        def Exit(self):
            global q
            q = 1

    if __name__ == "__main__" :
        app = QApplication(sys.argv)
        myWindow = WindowClass() 
        myWindow.show()
        app.exec_()
        
    return home()
        
if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2020 17:53:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2020 17:53:14] "GET /javascripts/video.js HTTP/1.1" 404 -
127.0.0.1 - - [22/Apr/2020 17:53:16] "GET /javascripts/video.js HTTP/1.1" 404 -
127.0.0.1 - - [22/Apr/2020 17:53:43] "POST /webcam/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2020 17:53:43] "GET /webcam/javascripts/video.js HTTP/1.1" 404 -
127.0.0.1 - - [22/Apr/2020 17:56:58] "POST /webcam/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2020 17:56:58] "GET /webcam/javascripts/video.js HTTP/1.1" 404 -


In [1]:
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
from tkinter import *
from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
import sys
from os import path
from flask import Flask

app = Flask(__name__)

@app.route('/') 

#웹페이지 템플릿
def home(): 
    return '''
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <title>Web Cam Video</title>
            <style>
            H1 {
              position: absolute;
              left: 100px;
              top: 15px;
              color:black;
            }
            </style>

            <H1>Webcam</H1>
            </head>
            <body>
                  <div class="booth">
                      <video id="video" width="400" height="300" autoplay=""></video>
                      <canvas id="canvas" width="400" height="300"></canvas>
            </div>
                  <script src="./javascripts/video.js"></script>
            </body>
            </html>

            <style>
            H2 {
              position: absolute;
              left: 120px;
              top: 100px;
              color:red;
            }
            </style>

            <H2><form action="/webcam/" method="post"><button name="forwardBtn" type="submit">webcam.exe</button></H2></form>'''

@app.route("/webcam/", methods=['POST'])

#메인함수
def Web_cam():
    #UI파일 연결
    #단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
    form_class = uic.loadUiType("controller.ui")[0]
    c = 0
    s = 0
    q = 0

    record = False
    rs = False
    r = False
    res = True

    #화면을 띄우는데 사용되는 Class 선언
    class WindowClass(QMainWindow, form_class) :

        def __init__(self) :

            super().__init__()
            self.setupUi(self)

            #버튼에 기능을 연결하는 코드
            self.pushButton.clicked.connect(self.Webcams)
            self.pushButton_2.clicked.connect(self.Backs)
            self.pushButton_3.clicked.connect(self.Nexts)
            self.pushButton_4.clicked.connect(self.Resets)
            self.pushButton_5.clicked.connect(self.Captures)
            self.pushButton_6.clicked.connect(self.Records)
            self.pushButton_7.clicked.connect(self.Stop_Records)
            self.pushButton_8.clicked.connect(self.Exit)
            
        #웹캠기능
        def Webcams(self):
            global c
            global s
            global q
            global record
            global rs
            global r
            global res

            q = 0
            c = 0
            s = 0
            
            record = False
            rs = False
            r = False
            res = True

            fourcc = cv2.VideoWriter_fourcc(*'XVID')

            logo = cv2.imread("./zero.png")

            cap = cv2.VideoCapture(0)

            cap.set(10000, 10000)

            time.sleep(3) #warming up
            if not cap.isOpened():
                exit()

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

            while True:    
                ret, image = cap.read()

                #현재시간
                now = datetime.datetime.now().strftime("%d_%H-%M-%S")
                key = cv2.waitKey(3)

                #대체 이미지
                if(res==True):
                    logo = cv2.imread("./zero.png")
                else:
                    if (c == 1): 
                        logo = cv2.imread("./pick.png")
                    elif (c == 2): 
                        logo = cv2.imread("./smile.png")
                    elif (c == 3):
                        logo = cv2.imread("./keobi.png")
                    elif (c == 4):
                        logo = cv2.imread("./hahoi.png")
                    elif (c == 5):
                        logo = cv2.imread("./trumph.png")
                    elif (c == 6):
                        logo = cv2.imread("./Pineapple.png")
                    elif (c == 7):
                        logo = cv2.imread("./Watermelon.png")
                    elif (c == 8):
                        logo = cv2.imread("./Lion.png")
                    elif (c == 9):
                        logo = cv2.imread("./Neo.png")
                    elif (c == 10):
                        logo = cv2.imread("./Momo.png")
                    elif (c == 11):
                        logo = cv2.imread("./Prodo.png")
                    elif (c == 12):
                        logo = cv2.imread("./Muzi.png")

                faces = face_cascade.detectMultiScale(image)

                for face in faces:
                    (x, y, w, h) = face
                    x1, y1, x2, y2 = x, y, x + w, y + h
                    #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)
                    
                    if(c == 0):
                        if(res == True):
                            logo = cv2.imread("./zero.png")
                        else:
                            image_face = image[y1:y2, x1:x2]
                            image_face = cv2.resize(image_face, (w//30, h//30))
                            image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                            image[y1:y2, x1:x2] = image_face
                            
                    else:
                        image_face = image[y1:y2, x1:x2]
                        logo = cv2.resize(logo, (w, h))
                        image_face =  cv2.add(image_face, logo)
                        image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                        image[y1:y2, x1:x2] = image_face

                    if(s == 1): 
                        cv2.imwrite("./" + str(now) + ".png", image)
                        s = 0

                    if(record == True):
                        record = False
                        r = True
                        video = cv2.VideoWriter("./" + str(now) + ".avi", fourcc, 30.0, (image.shape[1], image.shape[0]))

                    elif((r == True) and (rs == True)):
                        video.release()
                        record = False
                        rs = False
                        r = False

                    if(r == True):    
                        color = (0, 0, 255)
                        thickness = 2 
                        location = (550, 50)
                        font = cv2.FONT_HERSHEY_COMPLEX  # hand-writing style font
                        fontScale = 1.0
                        cv2.putText(image, 'REC', location, font, fontScale, color, thickness)
                        #print("녹화 중..")
                        video.write(image)


                    cv2.imshow("result", image)


                if (q == 1): #q
                    break

                if not ret:
                    break

            cap.release()
            cv2.destroyAllWindows()
        #다음 이미지
        def Nexts(self):
            global c
            if(c == 12):
                c = c
            else:
                c = c + 1
        #이전 이미지
        def Backs(self):
            global c
            if(c == 0):
                c = 0
            else:
                c = c - 1  
        #이미지 사용 여부
        def Resets(self):
            global res
            if(res == True):
                res = False
            else:
                res = True
        #캡쳐
        def Captures(self):
            global s
            s = 1
        #녹화 시작
        def Records(self):
            global record
            global r
            record = True
            r = True
        #녹화 중지
        def Stop_Records(self):
            global rs
            rs = True
            
        #웹캠 종료
        def Exit(self):
            global q
            q = 1

    if __name__ == "__main__" :
        app = QApplication(sys.argv)
        myWindow = WindowClass() 
        myWindow.show()
        app.exec_()
        
    return home()
        
if __name__ == "__main__" :
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Apr/2020 10:37:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2020 10:37:49] "GET /javascripts/video.js HTTP/1.1" 404 -
127.0.0.1 - - [22/Apr/2020 10:38:31] "POST /webcam/ HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2020 10:38:31] "GET /webcam/javascripts/video.js HTTP/1.1" 404 -


In [ ]:
#pyqt로 작성
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5 import QtCore, QtGui, QtWidgets 
import cv2
import time
import numpy as np
import datetime
from tkinter import *
from PyQt5 import QtCore
from PyQt5 import QtWidgets
from PyQt5 import QtGui
import sys
from os import path
from flask import Flask



#UI파일 연결
#단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
form_class = uic.loadUiType("controller.ui")[0]
c = 0
s = 0
q = 0

record = False
rs = False
r = False
res = True

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QDialog, form_class) :

    def __init__(self) :

        super().__init__()
        self.setupUi(self)

        #버튼에 기능을 연결하는 코드
        self.pushButton.clicked.connect(self.Webcams)
        self.pushButton_2.clicked.connect(self.Backs)
        self.pushButton_3.clicked.connect(self.Nexts)
        self.pushButton_4.clicked.connect(self.Resets)
        self.pushButton_5.clicked.connect(self.Captures)
        self.pushButton_6.clicked.connect(self.Records)
        self.pushButton_7.clicked.connect(self.Stop_Records)
        self.pushButton_8.clicked.connect(self.Exit)
        self.pushButton_9.clicked.connect(self.draw)

    def Webcams(self):
        global c
        global s
        global q
        global record
        global rs
        global r
        global res

        q = 0
        c = 0
        s = 0

        record = False
        rs = False
        r = False
        res = True

        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        logo = cv2.imread("./zero.png")

        #logo = cv2.resize(logo, (275,183))

        #logo_height, logo_width, _ = logo.shape

        cap = cv2.VideoCapture(0)

        cap.set(10000, 10000)

        #cap_height = cv2.CAP_PROP_FRAME_HEIGHT
        #cap_width =  cv2.CAP_PROP_FRAME_WIDTH

        time.sleep(3) #warming up
        if not cap.isOpened():
            exit()

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        while True:    
            ret, image = cap.read()

            #현재시간
            now = datetime.datetime.now().strftime("%d_%H-%M-%S")
            key = cv2.waitKey(3)

            #대체 이미지
            if(res==True):
                logo = cv2.imread("./zero.png")
            else:
                if c == 1: 
                    logo = cv2.imread("./pick.png")
                elif c == 2: 
                    logo = cv2.imread("./smile.png")
                elif c == 3:
                    logo = cv2.imread("./keobi.png")
                elif c == 4:
                    logo = cv2.imread("./hahoi.png")
                elif c == 5:
                    logo = cv2.imread("./trumph.png")

            faces = face_cascade.detectMultiScale(image)

            for face in faces:
                (x, y, w, h) = face
                x1, y1, x2, y2 = x, y, x + w, y + h
                #cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), thickness=2)

                if(c == 0):
                    if(res == True):
                        logo = cv2.imread("./zero.png")
                    else:
                        image_face = image[y1:y2, x1:x2]
                        image_face = cv2.resize(image_face, (w//30, h//30))
                        image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                        image[y1:y2, x1:x2] = image_face

                else:
                    image_face = image[y1:y2, x1:x2]
                    #logo = cv2.resize(logo, (y1-y2, x1-x2))
                    #image_face = logo[y1:y2, x1:x2]
                    logo = cv2.resize(logo, (w, h))
                    image_face =  cv2.add(image_face, logo)
                    image_face = cv2.resize(image_face, (w, h), interpolation=cv2.INTER_AREA)
                    image[y1:y2, x1:x2] = image_face

                if(s == 1): #z
                    #print("캡쳐")
                    cv2.imwrite("./" + str(now) + ".png", image)
                    s = 0

                if(record == True):
                    record = False
                    r = True
                    #print("녹화 시작")
                    video = cv2.VideoWriter("./" + str(now) + ".avi", fourcc, 30.0, (image.shape[1], image.shape[0]))

                elif((r == True) and (rs == True)):
                    #print("녹화 중지")
                    video.release()
                    record = False
                    rs = False
                    r = False

                if(r == True):    
                    color = (0, 0, 255)
                    thickness = 2 
                    location = (550, 50)
                    font = cv2.FONT_HERSHEY_COMPLEX  # hand-writing style font
                    fontScale = 1.0
                    cv2.putText(image, 'REC', location, font, fontScale, color, thickness)
                    #print("녹화 중..")
                    video.write(image)


                cv2.imshow("result", image)


            if (q == 1): #q
                break

            if not ret:
                break

            #image = cv2.add(image, logo)

        cap.release()
        cv2.destroyAllWindows()

    def Nexts(self):
        global c
        if(c == 5):
            c = c
        else:
            c = c + 1

    def Backs(self):
        global c
        if(c == 0):
            c = 0
        else:
            c = c - 1  

    def Resets(self):
        global res
        if(res == True):
            res = False
        else:
            res = True

    def Captures(self):
        global s
        s = 1

    def Records(self):
        global record
        global r
        record = True
        r = True

    def Stop_Records(self):
        global rs
        rs = True

    def Exit(self):
        global q
        q = 1          

if __name__ == "__main__" :
    app = QApplication(sys.argv)
    myWindow = WindowClass() 
    myWindow.show()
    app.exec_()
